In [0]:
import os
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Carregando a base de dados

In [0]:
data = pd.read_csv('sample_data/loan.csv')

In [3]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               601 non-null object
Married              611 non-null object
Dependents           599 non-null object
Education            614 non-null object
Self_Employed        582 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [5]:
data.Loan_Status.value_counts()

Y    422
N    192
Name: Loan_Status, dtype: int64

In [0]:
data2 = data[data.Loan_Status=='Y'].sample(200)

In [0]:
data = data2.append(data[data.Loan_Status=='N'].sample(192))

In [8]:
data.Loan_Status.value_counts()

Y    200
N    192
Name: Loan_Status, dtype: int64

## Checando Missing Values

In [9]:
data2.isnull().sum()

Loan_ID               0
Gender                4
Married               2
Dependents            4
Education             0
Self_Employed         3
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            4
Loan_Amount_Term      3
Credit_History       20
Property_Area         0
Loan_Status           0
dtype: int64

Preechendo Missing Values:

- `Dependents`: Assumindo o valor majoritário da coluna.
- `Self_Employed`: Assumindo o valor majoritário da coluna.
- `Loan_Amount_Term`: Preenchendo com o valor médio da coluna.
- `Credit_History`: Assumindo o valor majoritário da coluna.
- `Married`: Assumindo o valor majoritário da coluna.
- `Gender`: Assumindo o valor majoritário da coluna.

In [0]:
data['Gender'] = data['Gender'].fillna('Male')

In [0]:
data['Married'] = data['Married'].fillna('No')

In [0]:
data['Dependents'] = data['Dependents'].fillna('0')

In [0]:
data['Self_Employed'] = data['Self_Employed'].fillna('No')

In [0]:
data['LoanAmount'] = data['LoanAmount'].fillna(data['LoanAmount'].mean())

In [0]:
data['Credit_History'] = data['Credit_History'].fillna(1.0)

In [0]:
data['Loan_Amount_Term'] = data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mean())

In [17]:
data.Credit_History.value_counts()

1.0    306
0.0     86
Name: Credit_History, dtype: int64

### Checando novamente Missing Values

In [18]:
data.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

**Transformando dados categóricos**

Várias colunas do dataframe são categóricas, precisamos transforma-las, são elas: `Gender`, `Married`, `Education`, `Self_Employed` & `Property_Area` columns.

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
gender_values = {'Female' : 0, 'Male' : 1} 
married_values = {'No' : 0, 'Yes' : 1}
education_values = {'Graduate' : 0, 'Not Graduate' : 1}
employed_values = {'No' : 0, 'Yes' : 1}
dependent_values = {'3+': 3, '0': 0, '2': 2, '1': 1}
loan_values = {'Y':1,'N':0}
data.replace({'Gender': gender_values,
                 'Married': married_values, 
                 'Education': education_values,
                 'Self_Employed': employed_values, 
                 'Dependents': dependent_values,
                 'Loan_Status': loan_values
                }, inplace=True)

In [0]:
data.drop(['Loan_ID','CoapplicantIncome','Loan_Amount_Term','Credit_History','Property_Area'],axis=1,inplace=True)

In [22]:
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Status
443,1,0,1,0,0,37719,152.0,1
549,1,1,0,0,0,2785,110.0,1
544,0,1,0,1,0,3017,102.0,1
41,1,0,0,0,0,1800,47.0,1
123,1,1,2,0,0,2957,81.0,1


Selecionando o melhor classificador através de Pipeline e GridSearchCV

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
clf_rf = RandomForestClassifier(n_estimators=100,min_samples_split=2)

In [0]:
X_train = data.drop('Loan_Status',axis=1)
y = data['Loan_Status']

In [0]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X_train, y,test_size=0.20,random_state=42)

In [27]:
clf_rf.fit(X_treino,y_treino)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### Métricas de Validação ###

In [0]:
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [29]:
print (pd.crosstab(y_teste, clf_rf.predict(X_teste), rownames=['Real'], colnames=['Predito'], margins=True), '')

Predito   0   1  All
Real                
0        15  13   28
1        20  31   51
All      35  44   79 


In [30]:
print (metrics.classification_report(y_teste,clf_rf.predict(X_teste)))

              precision    recall  f1-score   support

           0       0.43      0.54      0.48        28
           1       0.70      0.61      0.65        51

    accuracy                           0.58        79
   macro avg       0.57      0.57      0.56        79
weighted avg       0.61      0.58      0.59        79



### Persistindo o modelo de Machine Learning para o disco. ###

In [0]:
from sklearn.externals import joblib

#### Persistindo o melhor modelo em disco.

In [50]:
joblib.dump(clf_rf, 'model.pkl')

['model.pkl']

#### Listando os arquivos em disco.

In [51]:
!ls

model.pkl  sample_data


#### Carregando o modelo a partir do disco para a memória.

In [0]:
model = joblib.load('model.pkl')

#### Verificando os atributos do modelo.

In [53]:
print("Atributos do Modelo:\n\nClasses:{}\n\nEstimators:{}\n\nParametros:{}".format(model.classes_,model.n_estimators,model.base_estimator))

Atributos do Modelo:

Classes:[0 1]

Estimators:100

Parametros:DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


**Verificando o Dataset final gerado.**

In [54]:
X_train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount
443,1,0,1,0,0,37719,152.0
549,1,1,0,0,0,2785,110.0
544,0,1,0,1,0,3017,102.0
41,1,0,0,0,0,1800,47.0
123,1,1,2,0,0,2957,81.0


**Teste de Classificação.**

In [0]:
teste = np.array([[1,1,3,0,0,9504,275.0]])

In [56]:
model.predict(teste)

array([1])

**Probabilidades de Classes.**

In [57]:
model.predict_proba(teste)

array([[0.47, 0.53]])